<a href="https://colab.research.google.com/github/OlivaVe/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [31]:
import time

In [6]:
# Import packages
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)

In [72]:
# Show DataFrame
df.show(1)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 1 row



In [12]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('temp_view_df')

In [16]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql('''
SELECT
  date_built            AS year,
  ROUND(AVG(price), 2)  AS average_price
FROM
  temp_view_df
WHERE
  bedrooms = 4
GROUP BY
  year
ORDER BY
year;
''').show()

+----+-------------+
|year|average_price|
+----+-------------+
|2010|    296800.75|
|2011|     302141.9|
|2012|    298233.42|
|2013|    299999.39|
|2014|    299073.89|
|2015|    307908.86|
|2016|    296050.24|
|2017|    296576.69|
+----+-------------+



In [19]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql('''
SELECT
  date_built            AS year,
  ROUND(AVG(price), 2)  AS average_price
FROM
  temp_view_df
WHERE
  bedrooms = 3    AND
  bathrooms = 3
GROUP BY
  year
ORDER BY
year;
''').show()

+----+-------------+
|year|average_price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [20]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql('''
SELECT
  date_built            AS year,
  ROUND(AVG(price), 2)  AS average_price
FROM
  temp_view_df
WHERE
  bedrooms = 3  AND
  bathrooms = 3 AND
  floors = 2    AND
  sqft_living >= 2000
GROUP BY
  year
ORDER BY
year;
''').show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    285010.22|
|2011|    276553.81|
|2012|    307539.97|
|2013|    303676.79|
|2014|    298264.72|
|2015|    297609.97|
|2016|     293965.1|
|2017|    280317.58|
+----+-------------+



In [57]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time= time.time()
spark.sql('''
SELECT
  view,
  ROUND(AVG(price), 2)    AS average_price
FROM
  temp_view_df
WHERE
  price >= 350000
GROUP BY
  view
ORDER BY
view;
''').show()
end_time= time.time()
real_time1 = end_time-start_time
print("--- %s seconds ---" % (real_time1))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|  10|    401868.43|
| 100|    1026669.5|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
|   2|    397389.25|
|  20|    399522.81|
|  21|    399758.88|
|  22|    402022.68|
|  23|    403411.92|
|  24|    400284.92|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

--- 0.641695499420166 seconds ---


In [58]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table temp_view_df")

DataFrame[]

In [59]:
# 8. Check if the table is cached.
spark.catalog.isCached('temp_view_df')

True

In [61]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
spark.sql('''
SELECT
  view,
  ROUND(AVG(price), 2)    AS average_price
FROM
  temp_view_df
WHERE
  price >= 350000
GROUP BY
  view
ORDER BY
view;
''').show()
end_time= time.time()
real_time2 = end_time-start_time
print("--- %s seconds ---" % (real_time2))


+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|  10|    401868.43|
| 100|    1026669.5|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
|   2|    397389.25|
|  20|    399522.81|
|  21|    399758.88|
|  22|    402022.68|
|  23|    403411.92|
|  24|    400284.92|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

--- 0.5658326148986816 seconds ---


In [62]:
print("---time uncached %s seconds ---" % (real_time1))
print("---time cached %s seconds ---" % (real_time2))

---time uncached 0.641695499420166 seconds ---
---time cached 0.5658326148986816 seconds ---


In [63]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_partition_home_sales")

In [64]:
# 11. Read the parquet formatted data.
parquet_df= spark.read.parquet('parquet_partition_home_sales')

In [65]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('temp_view_parquet')

In [68]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()
spark.sql('''
SELECT
  view,
  ROUND(AVG(price), 2)    AS average_price
FROM
  temp_view_parquet
WHERE
  price >= 350000
GROUP BY
  view
ORDER BY
view;
''').show()
end_time= time.time()
real_time_final = end_time-start_time
print("--- %s seconds ---" % (real_time_final))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|  10|    401868.43|
| 100|    1026669.5|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
|   2|    397389.25|
|  20|    399522.81|
|  21|    399758.88|
|  22|    402022.68|
|  23|    403411.92|
|  24|    400284.92|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

--- 0.8487069606781006 seconds ---


In [69]:
print("---time uncached:        %s seconds ---" % (real_time1))
print("---time cached:          %s seconds ---" % (real_time2))
print("---time cached parquet:  %s seconds ---" % (real_time_final))

---time uncached:        0.641695499420166 seconds ---
---time cached:          0.5658326148986816 seconds ---
---time cached parquet:  0.8487069606781006 seconds ---


In [70]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table temp_view_df")

DataFrame[]

In [71]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('temp_view_df')

False